---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [326]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, auc, accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder
pd.set_option('display.max_columns', 500)

In [294]:
dfTrain = pd.read_csv("readonly/train.csv", encoding = 'ISO-8859-1')
dfTest = pd.read_csv("readonly/test.csv", encoding = 'ISO-8859-1')
dfAddress = pd.read_csv('readonly/addresses.csv')
dfLatLon = pd.read_csv('readonly/latlons.csv')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [295]:
print(dfTrain.shape)
print(dfTest.shape)
print(dfAddress.shape)
print(dfLatLon.shape)

(250306, 34)
(61001, 27)
(311307, 2)
(121769, 3)


In [296]:
testlist = dfTest.columns
trainlist = dfTrain.columns
toRemove = []
for i in trainlist:
    if (i not in testlist) and i != "compliance":
        toRemove.append(i)

In [297]:
dfTrain = dfTrain.drop(toRemove, axis = 1)

In [298]:
ToRemoveFromAll = ['grafitti_status', "late_fee", "clean_up_cost", "judgment_amount", "grafitti_status",
                   "violation_description", "violation_zip_code", "non_us_str_code", "zip_code",
                  'agency_name', 'inspector_name', 'violator_name', 'mailing_address_str_number', 
                   'mailing_address_str_name', "state_fee", "violation_street_name", "ticket_issued_date",
                  "hearing_date", "violation_street_number"]

In [299]:
dfTrain = dfTrain.drop(ToRemoveFromAll, axis = 1)
dfTest = dfTest.drop(ToRemoveFromAll, axis = 1)

In [300]:
dfTrain.describe()

,ticket_id,fine_amount,admin_fee,discount_amount,compliance
count,250306.000000,250305.000000,250306.000000,250306.000000,159880.000000
mean,152665.543099,374.423435,12.774764,0.125167,0.072536
std,77189.882881,707.195807,9.607344,3.430178,0.259374
min,18645.000000,0.000000,0.000000,0.000000,0.000000
25%,86549.250000,200.000000,0.000000,0.000000,0.000000
50%,152597.500000,250.000000,20.000000,0.000000,0.000000
75%,219888.750000,250.000000,20.000000,0.000000,0.000000
max,366178.000000,10000.000000,20.000000,350.000000,1.000000


In [301]:
dfTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250306 entries, 0 to 250305
Data columns (total 10 columns):
ticket_id          250306 non-null int64
city               250306 non-null object
state              250213 non-null object
country            250306 non-null object
violation_code     250306 non-null object
disposition        250306 non-null object
fine_amount        250305 non-null float64
admin_fee          250306 non-null float64
discount_amount    250306 non-null float64
compliance         159880 non-null float64
dtypes: float64(4), int64(1), object(5)
memory usage: 19.1+ MB


In [302]:
dfTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250306 entries, 0 to 250305
Data columns (total 10 columns):
ticket_id          250306 non-null int64
city               250306 non-null object
state              250213 non-null object
country            250306 non-null object
violation_code     250306 non-null object
disposition        250306 non-null object
fine_amount        250305 non-null float64
admin_fee          250306 non-null float64
discount_amount    250306 non-null float64
compliance         159880 non-null float64
dtypes: float64(4), int64(1), object(5)
memory usage: 19.1+ MB


In [303]:
# dfTrain = dfTrain.drop(dfTrain[dfTrain["hearing_date"].isnull()].index)
dfTrain = dfTrain.drop(dfTrain[dfTrain["state"].isnull()].index)
dfTrain["compliance"].fillna(-1, inplace = True)
dfTrain['fine_amount'].fillna(-1, inplace = True)

In [304]:
dfTrain.index = dfTrain.ticket_id
dfTrain = dfTrain.drop("ticket_id", axis = 1)
dfTest.index = dfTest.ticket_id
dfTest = dfTest.drop("ticket_id", axis = 1) 
dfAddress.index = dfAddress.ticket_id
dfAddress = dfAddress.drop("ticket_id", axis = 1)

In [305]:
labelencoder = LabelEncoder()
dfTrain.city = labelencoder.fit_transform(dfTrain.city)
dfTrain.state = labelencoder.fit_transform(dfTrain.state)
dfTrain.country = labelencoder.fit_transform(dfTrain.country)
dfTrain.violation_code = labelencoder.fit_transform(dfTrain.violation_code)
dfTrain.disposition = labelencoder.fit_transform(dfTrain.disposition)
dfTrain.violation_code = labelencoder.fit_transform(dfTrain.violation_code)
# dfTrain['ticket_issued_date'] = pd.to_datetime(dfTrain['ticket_issued_date']) 
# dfTrain['hearing_date'] = pd.to_datetime(dfTrain['hearing_date'])




In [306]:
dfTrain.iloc[:, 1:-1].columns
dfTrain.iloc[:, -1:].columns

Index(['compliance'], dtype='object')

In [307]:
X_train, X_test, y_train, y_test = train_test_split(dfTrain.iloc[:, 0:-1], dfTrain.iloc[:, -1:], 
                                                    test_size=0.25, 
                                                    random_state=42)

In [308]:
X_train.columns

Index(['city', 'state', 'country', 'violation_code', 'disposition',
       'fine_amount', 'admin_fee', 'discount_amount'],
      dtype='object')

In [309]:
knn = DecisionTreeClassifier()
knn.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [317]:
y_pred = knn.predict(X_test)

In [354]:
knn.score(y_tes, y_pr)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


ValueError: Number of features of the model must match the input. Model n_features is 8 and input n_features is 39721 

In [351]:
ytest = y_test.iloc[:, 0]

In [352]:
y_pr = y_pred[y_pred >= 0]
y_tes = ytest[ytest >= 0]

In [353]:
roc_auc_score(y_tes, y_pr)

0.58154196400577884

array([ 0.,  0., -1., ...,  0.,  0., -1.])

In [357]:
import pandas as pd
import numpy as np
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

def blight_model():
    
    # Your code here
    
    df = pd.read_csv('readonly/train.csv', encoding = "ISO-8859-1")
    
    df.index = df['ticket_id']
    
#     features_name = ['agency_name', 'inspector_name', 'violator_name', 'violation_street_number', 
#                      'violation_street_name', 'mailing_address_str_number', 'mailing_address_str_name',
#                      'city', 'state', 'zip_code', 'ticket_issued_date', 'hearing_date',
#                      'violation_code', 'violation_description', 'disposition', 'fine_amount', 'admin_fee',
#                      'state_fee' , 'late_fee', 'discount_amount', 'clean_up_cost' , 'judgment_amount'
#                      ]


    features_name = ['fine_amount', 'admin_fee', 'state_fee', 'late_fee']
        
    df.compliance = df.compliance.fillna(value=-1)
    
    df = df[df.compliance != -1]
    
#     le = LabelEncoder().fit(df['inspector_name'])
    
#     inspector_name_transformed = le.transform(df['inspector_name'])
 
    
    X = df[features_name]
    
#     X['inspector_name'] = le.transform(df['inspector_name'])
    
#     print(X)
    
    X.fillna(value = -1)
    
    y = df.compliance
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
    
    clf = RandomForestClassifier(n_estimators = 10, max_depth = 5).fit(X_train, y_train)
    
#     grid_values = {'n_estimators': [9, 10, 11], 'max_depth': [1,2,3,4,5] }  # n_est = 10 and max_depth = 5
    
    # default metric to optimize over grid parameters: accuracy
#     grid_clf = GridSearchCV(clf, param_grid = grid_values)
#     grid_clf.fit(X_train, y_train)

    
#     y_score = clf.predict(X_test)
    
#     fpr, tpr, _ = roc_curve(y_test, y_score)
    
#     roc_auc = auc(fpr, tpr)
    
#     print(roc_auc)

    features_name = ['fine_amount', 'admin_fee', 'state_fee', 'late_fee']
    
    df_test = pd.read_csv('readonly/test.csv', encoding = "ISO-8859-1")
    
    df_test.index = df_test['ticket_id']
    
    X_predict = clf.predict_proba(df_test[features_name])
    
    ans = pd.Series(data = X_predict[:,1], index = df_test['ticket_id'], dtype='float32')

#     print(ans)
    
    return ans

blight_model()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


ticket_id
284932    0.060342
285362    0.027013
285361    0.068724
285338    0.060342
285346    0.068724
285345    0.060342
285347    0.056346
285342    0.413340
285530    0.027013
284989    0.028436
285344    0.056346
285343    0.027013
285340    0.027013
285341    0.056346
285349    0.068724
285348    0.060342
284991    0.028436
285532    0.028436
285406    0.028436
285001    0.028436
285006    0.027013
285405    0.027013
285337    0.028436
285496    0.056346
285497    0.060342
285378    0.027013
285589    0.028436
285585    0.060342
285501    0.068724
285581    0.027013
            ...   
376367    0.028436
376366    0.035901
376362    0.035901
376363    0.060342
376365    0.028436
376364    0.035901
376228    0.035901
376265    0.035901
376286    0.366631
376320    0.035901
376314    0.035901
376327    0.366631
376385    0.366631
376435    0.477828
376370    0.366631
376434    0.056346
376459    0.068724
376478    0.008271
376473    0.035901
376484    0.023940
376482    0.028436
37

In [ ]:
blight_model()